In [44]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

#regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statistics import *
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

#clustering
import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn import metrics

In [45]:
prec = pd.read_csv('precipitacionsbarcelonadesde1786.csv')
prec.head()

,Any,Precip_Acum_Gener,Precip_Acum_Febrer,Precip_Acum_Marc,Precip_Acum_Abril,Precip_Acum_Maig,Precip_Acum_Juny,Precip_Acum_Juliol,Precip_Acum_Agost,Precip_Acum_Setembre,Precip_Acum_Octubre,Precip_Acum_Novembre,Precip_Acum_Desembre
0,1786,32.8,28.4,84.4,42.3,8.5,38.0,6.8,52.1,62.0,57.0,195.8,114.1
1,1787,136.4,27.4,44.6,76.9,0.0,29.8,34.7,14.9,205.8,12.4,42.2,0.0
2,1788,9.9,14.9,32.2,12.4,7.5,146.3,59.4,29.8,123.9,163.6,106.6,22.3
3,1789,12.4,12.4,23.6,9.9,32.2,9.9,6.3,69.4,18.7,52.1,18.7,76.9
4,1790,44.6,1.2,188.5,74.4,44.7,39.7,65.7,32.3,71.9,205.8,72.0,14.9


In [46]:
#rename columns
prec.rename(columns={'Precip_Acum_Gener': "Gener",
                   'Precip_Acum_Febrer': "Febrer",
                   'Precip_Acum_Marc': "Març",
                   'Precip_Acum_Abril': "Abril",
                   'Precip_Acum_Maig': "Maig",
                   'Precip_Acum_Juny': "Juny",
                   'Precip_Acum_Juliol': "Juliol",
                   'Precip_Acum_Agost': "Agost",
                   'Precip_Acum_Setembre': "Setembre",
                   'Precip_Acum_Octubre': "Octubre",
                   'Precip_Acum_Novembre': "Novembre",
                   'Precip_Acum_Desembre': "Desembre"},inplace=True)

In [47]:
prec.columns

Index(['Any', 'Gener', 'Febrer', 'Març', 'Abril', 'Maig', 'Juny', 'Juliol',
       'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre'],
      dtype='object')

In [48]:
years = prec['Any'].unique()
df = pd.DataFrame(years, columns = ['Any'])
df['id'] = prec.index + 1
df.head(2)

,Any,id
0,1786,1
1,1787,2


In [49]:
df = prec.merge(df, how='left',on='Any')

In [50]:
df

,Any,Gener,Febrer,Març,Abril,Maig,Juny,Juliol,Agost,Setembre,Octubre,Novembre,Desembre,id
0,1786,32.8,28.4,84.4,42.3,8.5,38.0,6.8,52.1,62.0,57.0,195.8,114.1,1
1,1787,136.4,27.4,44.6,76.9,0.0,29.8,34.7,14.9,205.8,12.4,42.2,0.0,2
2,1788,9.9,14.9,32.2,12.4,7.5,146.3,59.4,29.8,123.9,163.6,106.6,22.3,3
3,1789,12.4,12.4,23.6,9.9,32.2,9.9,6.3,69.4,18.7,52.1,18.7,76.9,4
4,1790,44.6,1.2,188.5,74.4,44.7,39.7,65.7,32.3,71.9,205.8,72.0,14.9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2018,63.6,104.6,112.5,83.0,30.3,53.1,51.5,36.5,44.4,201.9,201.8,4.8,233
233,2019,18.2,4.6,0.3,36.2,109.0,19.3,75.8,23.1,71.5,83.2,39.4,119.2,234
234,2020,89.3,2.8,61.6,258.7,29.2,69.1,23.1,12.4,60.2,23.1,52.5,41.5,235
235,2021,26.2,26.5,7.7,62.7,13.4,7.8,9.0,8.3,23.3,63.2,75.9,3.8,236


In [51]:
# Creates a pivot table dataframe
table = df.melt(id_vars=['Any', 'id'], value_vars=['Gener', 'Febrer', 'Març', 'Abril', 'Maig', 'Juny', 'Juliol',
       'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre'])


In [52]:
table.rename(columns = {'value': "Total_precipitació"})

,Any,id,variable,Total_precipitació
0,1786,1,Gener,32.8
1,1787,2,Gener,136.4
2,1788,3,Gener,9.9
3,1789,4,Gener,12.4
4,1790,5,Gener,44.6
...,...,...,...,...
2839,2018,233,Desembre,4.8
2840,2019,234,Desembre,119.2
2841,2020,235,Desembre,41.5
2842,2021,236,Desembre,3.8


In [53]:
temp = pd.read_csv('temperaturesbarcelonadesde1780.csv')
temp.head()

,Any,Temp_Mitjana_Gener,Temp_Mitjana_Febrer,Temp_Mitjana_Marc,Temp_Mitjana_Abril,Temp_Mitjana_Maig,Temp_Mitjana_Juny,Temp_Mitjana_Juliol,Temp_Mitjana_Agost,Temp_Mitjana_Setembre,Temp_Mitjana_Octubre,Temp_Mitjana_Novembre,Temp_Mitjana_Desembre
0,1786,7.8,8.3,9.9,12.8,16.8,20.2,20.9,21.1,19.5,15.5,9.2,8.3
1,1787,5.4,7.8,11.3,12.1,14.7,19.2,21.0,21.8,19.4,15.5,9.3,10.8
2,1788,6.4,10.1,10.4,12.5,17.1,19.2,22.1,23.0,19.1,15.5,10.5,5.4
3,1789,6.9,9.3,8.7,13.3,17.7,18.5,20.9,21.9,20.0,14.7,7.8,7.3
4,1790,7.4,9.5,10.4,12.3,15.0,19.0,20.8,23.1,19.3,17.1,11.7,7.9


In [54]:
#rename columns
temp.rename(columns={'Temp_Mitjana_Gener': "Gener",
                   'Temp_Mitjana_Febrer': "Febrer",
                   'Temp_Mitjana_Marc': "Març",
                   'Temp_Mitjana_Abril': "Abril",
                   'Temp_Mitjana_Maig': "Maig",
                   'Temp_Mitjana_Juny': "Juny",
                   'Temp_Mitjana_Juliol': "Juliol",
                   'Temp_Mitjana_Agost': "Agost",
                   'Temp_Mitjana_Setembre': "Setembre",
                   'Temp_Mitjana_Octubre': "Octubre",
                   'Temp_Mitjana_Novembre': "Novembre",
                   'Temp_Mitjana_Desembre': "Desembre"},inplace=True)

In [60]:
years = temp['Any'].unique()
df = pd.DataFrame(years, columns = ['Any'])
df['id'] = prec.index + 1
df.head(2)

,Any,id
0,1786,1
1,1787,2


In [61]:
df = temp.merge(df, how='left',on='Any')
df

,Any,Gener,Febrer,Març,Abril,Maig,Juny,Juliol,Agost,Setembre,Octubre,Novembre,Desembre,id
0,1786,7.8,8.3,9.9,12.8,16.8,20.2,20.9,21.1,19.5,15.5,9.2,8.3,1
1,1787,5.4,7.8,11.3,12.1,14.7,19.2,21.0,21.8,19.4,15.5,9.3,10.8,2
2,1788,6.4,10.1,10.4,12.5,17.1,19.2,22.1,23.0,19.1,15.5,10.5,5.4,3
3,1789,6.9,9.3,8.7,13.3,17.7,18.5,20.9,21.9,20.0,14.7,7.8,7.3,4
4,1790,7.4,9.5,10.4,12.3,15.0,19.0,20.8,23.1,19.3,17.1,11.7,7.9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2018,10.5,6.7,10.8,14.7,17.1,21.5,25.3,25.8,22.5,17.0,12.4,11.1,233
233,2019,8.1,11.9,13.5,13.4,15.6,21.9,25.4,25.1,21.8,18.5,11.9,11.2,234
234,2020,10.0,12.8,11.9,14.3,19.4,20.1,25.0,25.5,21.7,16.4,14.7,9.3,235
235,2021,7.7,11.6,12.1,12.9,17.3,23.3,24.8,24.5,23.0,18.1,11.3,10.9,236


In [62]:
# Creates a pivot table dataframe
table1 = temp.melt(id_vars=['Any','id'], value_vars=['Gener', 'Febrer', 'Març', 'Abril', 'Maig', 'Juny', 'Juliol',
       'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre'])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['id']"

In [ ]:
table1.rename(columns = {'value': "Total_temperatures"})

In [ ]:
pd.merge(clients, invoices, on='client_id')